In [ ]:
# only for Google Colab
!mkdir data
!mkdir checkpoints

!wget -O data/data.zip 'https://docs.google.com/uc?export=download&id=188n0P4jQ7J40oH930cwLWXwOVzvjS2vn' --no-check-certificate
!unzip data/* -d data/

In [3]:
import load_files
import metrics
import xy_normalizer
import tfm


d_num_layers = 6  # Price, Ordered volume, Filled volume, Canceled volume, Pending volume, Time index
d_num_price_levels = 10 * 2 * 2  # price level ($10) per 50 cents per level (*2) per side (*2)
d_minutes_per_day = int(6.5 * 60)  # 6 hours 30 minutes of data per trading session, from 9:30 to 16:00

d = load_files.load_d(d_num_layers, d_num_price_levels, d_minutes_per_day)

x_block_length = 10 # in minutes
y_block_length = 1 # in minutes
(X, y) = xy_normalizer.make_x_and_y(d, x_block_length, y_block_length)
(X, y) = xy_normalizer.normalize_data(X, y)

train_data_pointer = len(X) - int(len(X) * 0.2) # 20%
(X_train, y_train, X_test, y_test) = xy_normalizer.make_train_and_test_sets(X, y, train_data_pointer)

KeyboardInterrupt: 

In [ ]:
batch_size = 8192
epochs = 10000

model = tfm.create_model()
model = tfm.fit_model(model, batch_size, epochs, X_train, y_train, X_test, y_test)

In [ ]:
import pandas as pd

pd.DataFrame(model.history.history).plot()

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
model = tfm.load_best_weights(model)

In [ ]:
predictions = model.predict(X_test)
metrics.get_accuracy(y_test, predictions)
metrics.get_mean_absolute_error(y_test, predictions)